In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import itertools
import re
from itertools import product
import operator
from functools import reduce
import polars as pl
#from pandas import option_context
#%pip install openpyxl
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-pastel')
plt.style.use('seaborn-dark')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/2875002457.py:15: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-pastel')
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/2875002457.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-dark')


In [1]:
import matplotlib
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
matplotlib.style.use('dark_background')
color_list = ['white','skyblue','pink', 'tomato', 'slateblue', 'wheat', 'lightgrey', 'limegreen', 'royalblue', 'yellowgreen', 'lightseagreen', 'orchid', 'orange']
cmap = LinearSegmentedColormap.from_list("",color_list)

In [2]:
# Import all the google connexion packages
import os
import google.auth
from google.cloud import bigquery
from google.auth import impersonated_credentials

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/a20015173/.config/gcloud/legacy_credentials/ekaterina.rezanovich@adeo.com/adc.json"
client_1 = bigquery.Client(project = 'ccdp-transverse-data-dev')

NPS DATA

In [3]:
# questions data
filepath_1 = "./datasets/questions.parquet"
if not os.path.isfile(filepath_1):

    query = """
    select t1.survey_id, t1.name , t2.question_id, t2.name, t2.content
    from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_survey` as t1
    left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_question` as t2 on t2.survey_id = t1.survey_id
    where t2.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
    'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')"""

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_1)

df_q = pd.read_parquet(filepath_1)

In [4]:
# answers data
filepath_2 = "./datasets/answers.parquet"
if not os.path.isfile(filepath_2):

    query = """
select t3.answer_id, t3.feedback_id, t3.question_id, t3.score, t3.score_category, t3.comment,
t4.touchpoint_id, t4.store_code, t4.user_id, t4.user_id_type, cast(t4.date as date) as answer_date
from `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_answer` as t3
left join `dfdp-customer-exp-mt-prod.customer_experience_measure_bu001.client_experience_feedback` as t4 on t4.feedback_id = t3.feedback_id
where t3.question_id in ('SV_81jKyfOtepO6Weq__QID1_NPS_GROUP', 'SV_4OUB8z9FwEgBiU6__QID1_NPS_GROUP', 'SV_5nFeSHRTsx1yxWC__QID1_NPS_GROUP', 'SV_77FLHcnyIyMM07Q__QID1_NPS_GROUP', 'SV_bj9J6UgbM2yIATQ__QID13_NPS_GROUP',
'SV_2fA21joHLvPEZoi__QID1_NPS_GROUP', 'SV_5nVqXD824PZXj14__QID1_NPS_GROUP')
    """

    dl_data = pd.read_gbq(query=query, project_id="ccdp-transverse-data-dev")
    dl_data.to_parquet(filepath_2)

df_a = pd.read_parquet(filepath_2)

In [5]:
df_nps_all = df_a.merge(df_q, left_on = 'question_id', right_on = 'question_id', how='left')

In [6]:
del df_nps_all['name_1'], df_nps_all['user_id_type'],  df_nps_all['content']

In [7]:
# filter only nps with user_id
df_nps = df_nps_all[df_nps_all['user_id'].notna()]

In [8]:
# first and last TP of the customer
df_nps_dates = df_nps.groupby('user_id')['answer_date'].agg(['min', 'max']).reset_index()
df_nps_dates.columns = ['uid', 'first_nps_date', 'last_nps_date']

In [9]:
df_nps_1 = df_nps.merge(df_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_nps_1['uid']

In [10]:
# nb FB of the customer
df_nps_nb_fb = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
df_nps_nb_fb.columns = ['uid', 'nb_fb']

In [11]:
df_nps_2 = df_nps_1.merge(df_nps_nb_fb, left_on='user_id', right_on='uid', how='left')
del df_nps_2['uid']

In [12]:
# 0 for detractor, 1 for passive, 2 for promotor
def fb_type(df):
    if df['score'] <= 6 :
        return 0
    elif df['score'] >= 9:
        return 2
    else:
        return 1

In [13]:
df_nps_2['score'].unique()

<IntegerArray>
[8, 10, 9, 4, 0, 3, 6, 5, 1, 7, 2, <NA>]
Length: 12, dtype: Int64

In [14]:
df_nps_3 = df_nps_2[df_nps_2['score'].notna()]

In [15]:
df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/761178500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_3['fb_type'] = df_nps_3.apply(fb_type, axis=1)


In [16]:
# types of the customer fb
df_nps_fb_typ = df_nps_3.pivot_table(index='user_id', columns='fb_type', values='feedback_id', aggfunc='nunique').reset_index()
df_nps_fb_typ.columns = ['uid', '0', '1', '2']
df_nps_fb_typ = df_nps_fb_typ.fillna(0)
df_nps_fb_typ.head()

,uid,0,1,2
0,1000000,0.0,0.0,2.0
1,10000024,1.0,0.0,0.0
2,1000004,0.0,0.0,1.0
3,1000011,2.0,0.0,0.0
4,10000188,0.0,0.0,1.0


In [17]:
def fb_type_res(df):
    if df['0']==0 and df['1']==0:
        return "only positive"
    elif df['0']==0 and df['2']==0:
        return "only passive"
    elif df['1']==0 and df['2']==0:
        return "only negative"
    elif df['0']==0:
        return "passive and positive"
    elif df['1']==0:
        return "negative and positive"  
    elif df['2']==0:
        return "negative and passive"  
    else:
        return "negative, passive, positive"

In [18]:
df_nps_fb_typ['fb_type'] = df_nps_fb_typ.apply(fb_type_res, axis=1)

In [19]:
def fb_type_res_nb(df):
    if (df['0']+df['1']+df['2']) > 1:
        return "mutli"
    else:
        return "mono"

In [20]:
df_nps_fb_typ['fb_nb_type'] = df_nps_fb_typ.apply(fb_type_res_nb, axis=1)

In [21]:
df_nps_fb_typ.columns = ['uid', 'nb_fb_neg', 'nb_fb_pas', 'nb_fb_pos', 'fb_group', 'fb_nb_type']

In [22]:
df_nps_4 = df_nps_3.merge(df_nps_fb_typ, left_on='user_id', right_on='uid', how='left')
del df_nps_4['uid']

In [23]:
df_nps_4.head(2)

,answer_id,feedback_id,question_id,score,score_category,comment,touchpoint_id,store_code,user_id,answer_date,...,name,first_nps_date,last_nps_date,nb_fb,fb_type,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,R_111w3BVzUYwknmY__QID1_NPS_GROUP,R_111w3BVzUYwknmY,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,8,Passif,"Bonjour, \nJe suis entièrement satisfait de ...",SV_81jKyfOtepO6Weq,<NA>,10217876,2023-08-20,...,LMFR - Client SAV - PROD,2022-04-09,2023-08-20,14,1,0.0,1.0,13.0,passive and positive,mutli
1,R_3htf0IgZ0ecDb8u__QID1_NPS_GROUP,R_3htf0IgZ0ecDb8u,SV_81jKyfOtepO6Weq__QID1_NPS_GROUP,10,Promoteur,None,SV_81jKyfOtepO6Weq,<NA>,54974624,2023-08-16,...,LMFR - Client SAV - PROD,2023-08-16,2023-08-16,1,2,0.0,0.0,1.0,only positive,mono


In [24]:
df_nps_summary_type = df_nps_4.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,889117,72.614886
1,mutli,335311,27.385114


In [25]:
df_nps_summary_group = df_nps_4.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,680303,55.560882
4,only passive,277548,22.667564
3,only negative,129318,10.561503
6,passive and positive,77828,6.356274
1,negative and positive,26855,2.193269
0,negative and passive,21942,1.792020
2,"negative, passive, positive",10634,0.868487


In [26]:
df_nps_summary = df_nps_4.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,524428,42.830448
1,mono,only passive,246769,20.153819
0,mono,only negative,117920,9.630619
8,mutli,only positive,155875,12.730434
9,mutli,passive and positive,77828,6.356274
7,mutli,only passive,30779,2.513745
4,mutli,negative and positive,26855,2.193269
3,mutli,negative and passive,21942,1.792020
6,mutli,only negative,11398,0.930884
5,mutli,"negative, passive, positive",10634,0.868487


SALES DATA

In [27]:
df_s0 = pd.read_parquet('./datasets/sales_gr/df_s_3p.parquet')
df_s1 = pd.read_parquet('./datasets/sales_gr/df_s_22_1_1p.parquet')

In [28]:
df_s2 = pd.read_parquet('./datasets/sales_gr/df_s_22_2_1p.parquet')

In [29]:
df_s3 = pd.read_parquet('./datasets/sales_gr/df_s_22_3_1p.parquet')

In [30]:
df_s4 = pd.read_parquet('./datasets/sales_gr/df_s_22_4_1p.parquet')

In [31]:
df_s5 = pd.read_parquet('./datasets/sales_gr/df_s_23_1_1p.parquet')

In [32]:
df_s6 = pd.read_parquet('./datasets/sales_gr/df_s_23_2_1p.parquet')

In [33]:
df_s7 = pd.read_parquet('./datasets/sales_gr/df_s_23_3_1p.parquet')

In [34]:
df_s_1p = df_s0.append([df_s1,df_s2,df_s3])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/2518559154.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_1p = df_s0.append([df_s1,df_s2,df_s3])


In [35]:
df_s_2p = df_s4.append([df_s5,df_s6,df_s7])

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/3628812971.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s_2p = df_s4.append([df_s5,df_s6,df_s7])


In [36]:
df_s = df_s_1p.append(df_s_2p)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/4204960968.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_s = df_s_1p.append(df_s_2p)


In [37]:
df_s.head()

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty
0,3,2023-09-01,3#13#6281#2023-09-01,95443306,1,Store,Self-service,-165.60,-1
1,3,2023-09-01,3#13#6282#2023-09-01,61918500,1,Store,Self-service,-166.70,-3
2,3,2023-09-01,3#13#6284#2023-09-01,2416854,1,Store,Self-service,-59.18,-4
3,3,2023-09-01,3#13#6285#2023-09-01,52538909,1,Store,Self-service,-103.29,-3
4,3,2023-09-01,3#13#6287#2023-09-01,56376378,1,Store,Self-service,-71.80,-2


In [38]:
df_s['cart_date'].min()

datetime.date(2022, 1, 1)

In [39]:
df_s['cart_date'].max()

datetime.date(2023, 10, 19)

In [40]:
df_s['store_id'].unique()

<IntegerArray>
[  3,   4,   5,   6,  10,  11,  12,  13,  14,  15,
 ...
 291, 292, 293, 294, 295, 296, 297, 298, 380, 230]
Length: 146, dtype: Int64

In [41]:
df_s['sales_channel'].unique()

array(['Store', 'Website'], dtype=object)

In [42]:
df_s['delivery_channel'].unique()

array(['Self-service', 'Pickup', 'Self-service - Sales Bulletin',
       'Delivery'], dtype=object)

In [43]:
no_user_id = ['undefined', 'a', 'ra']
df_nps = df_nps.query('user_id not in @no_user_id')
df_nps['user_id'] = df_nps['user_id'].astype('int')

In [44]:
df_nps_users = df_nps.groupby('user_id')['feedback_id'].nunique().reset_index()
nps_users = df_nps_users['user_id'].to_list()

In [45]:
# filter users with nps reviews
df_s_nps = df_s.query('user_id in @nps_users')

In [46]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_nps['user_id'].nunique())

Nb customers with purchase 8419907
Nb customers with purchase and nps 1097904


In [47]:
# first,last dates of purchases by channel
df_s_nps_ch_dates = df_s_nps.pivot_table(index='user_id', columns = 'sales_channel',values='cart_date',\
                                          aggfunc=['min','max']).reset_index()
df_s_nps_ch_dates.columns = ['uid', 'min_date_store', 'min_date_web', 'max_date_store', 'max_date_web']
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [48]:
# first,last dates of purchases in general
df_s_nps_dates = df_s_nps.groupby('user_id').agg({"cart_date": ['min', 'max']}).reset_index()
df_s_nps_dates.columns = ['uid', 'min_date', 'max_date']
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [49]:
#merge this info to sales table
df_s_2 = df_s_nps.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')
del df_s_2['uid']
df_s_2.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web
0,3,2023-09-01,3#13#6284#2023-09-01,2416854,1,Store,Self-service,-59.18,-4,2022-04-04,2022-09-29,2023-09-01,2023-10-04
1,3,2023-09-01,3#13#6288#2023-09-01,62771021,1,Store,Self-service,-58.60,-10,2022-01-06,2022-05-09,2023-10-10,2023-09-15


In [50]:
df_s_3 = df_s_2.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')
del df_s_3['uid']
df_s_3.head(2)

,store_id,cart_date,cart_id,user_id,top_l_c,sales_channel,delivery_channel,amount_to,qty,min_date_store,min_date_web,max_date_store,max_date_web,min_date,max_date
0,3,2023-09-01,3#13#6284#2023-09-01,2416854,1,Store,Self-service,-59.18,-4,2022-04-04,2022-09-29,2023-09-01,2023-10-04,2022-04-04,2023-10-04
1,3,2023-09-01,3#13#6288#2023-09-01,62771021,1,Store,Self-service,-58.60,-10,2022-01-06,2022-05-09,2023-10-10,2023-09-15,2022-01-06,2023-10-10


In [51]:
df_nps_5 = df_nps_4.query('user_id not in @no_user_id')
df_nps_5['uid'] = df_nps_5['user_id'].astype('int')

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/692595386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nps_5['uid'] = df_nps_5['user_id'].astype('int')


In [52]:
df_nps_6 = df_nps_5[['uid','first_nps_date', 'last_nps_date', 'nb_fb', 'nb_fb_neg', 'nb_fb_pas','nb_fb_pos','fb_group',\
                                 'fb_nb_type']]

In [55]:
df_nps_6.head(2)

,uid,first_nps_date,last_nps_date,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,10217876,2022-04-09,2023-08-20,14,0.0,1.0,13.0,passive and positive,mutli
1,54974624,2023-08-16,2023-08-16,1,0.0,0.0,1.0,only positive,mono


In [57]:
# users' purchases characterisrics
df_s_ch = df_s_3.pivot_table(index=['user_id', 'top_l_c'], columns='sales_channel', 
                             aggfunc={'amount_to' : 'sum', 'qty' : 'sum', 'cart_id':'nunique'}).reset_index()

In [64]:
df_s_ch.columns = ['user_id','top_l_c','amount_to_store','amount_to_web','nb_carts_store', 'nb_carts_web', 'qty_store','qty_web']
df_s_ch.head(2)

,user_id,top_l_c,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web
0,14,1,381.48,341.01,16.0,4.0,36.0,9.0
1,15,1,6672.94,940.28,59.0,10.0,340.0,110.0


In [62]:
df_s_nps_ch_dates.head(2)

,uid,min_date_store,min_date_web,max_date_store,max_date_web
0,14,2022-02-12,2022-03-08,2023-09-05,2023-07-15
1,15,2022-02-09,2022-01-03,2023-10-07,2023-10-10


In [63]:
df_s_nps_dates.head(2)

,uid,min_date,max_date
0,14,2022-02-12,2023-09-05
1,15,2022-01-03,2023-10-10


In [60]:
df_nps_6.head(2)

,uid,first_nps_date,last_nps_date,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,10217876,2022-04-09,2023-08-20,14,0.0,1.0,13.0,passive and positive,mutli
1,54974624,2023-08-16,2023-08-16,1,0.0,0.0,1.0,only positive,mono


In [65]:
df_s_4 = df_s_ch.merge(df_s_nps_ch_dates, left_on='user_id', right_on='uid', how='left')

In [66]:
df_s_5 = df_s_4.merge(df_s_nps_dates, left_on='user_id', right_on='uid', how='left')

In [67]:
df_s_6 = df_s_5.merge(df_nps_6, left_on='user_id', right_on='uid', how='left')

In [70]:
del df_s_6['uid_x'], df_s_6['uid_y'], df_s_6['uid']

In [71]:
df_s_6.head()

,user_id,top_l_c,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,...,min_date,max_date,first_nps_date,last_nps_date,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type
0,14,1,381.48,341.01,16.0,4.0,36.0,9.0,2022-02-12,2022-03-08,...,2022-02-12,2023-09-05,2022-05-04,2022-05-04,1.0,0.0,1.0,0.0,only passive,mono
1,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2022-01-03,2023-10-10,2022-11-13,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli
2,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2022-01-03,2023-10-10,2022-11-13,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli
3,16,1,2394.90,NaN,51.0,NaN,162.0,NaN,2022-02-14,NaT,...,2022-02-14,2023-10-11,2023-02-07,2023-02-07,1.0,0.0,0.0,1.0,only positive,mono
4,19,1,4834.76,NaN,31.0,NaN,222.0,NaN,2022-05-05,NaT,...,2022-05-05,2023-10-19,2022-06-29,2022-11-15,2.0,0.0,1.0,1.0,passive and positive,mutli


In [73]:
df_s_6.to_parquet('.\datasets\sales_summary_nps.parquet')

In [74]:
df_s_6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1869095 entries, 0 to 1869094
Data columns (total 22 columns):
 #   Column           Dtype  
---  ------           -----  
 0   user_id          Int64  
 1   top_l_c          Int64  
 2   amount_to_store  float64
 3   amount_to_web    float64
 4   nb_carts_store   float64
 5   nb_carts_web     float64
 6   qty_store        float64
 7   qty_web          float64
 8   min_date_store   dbdate 
 9   min_date_web     dbdate 
 10  max_date_store   dbdate 
 11  max_date_web     dbdate 
 12  min_date         dbdate 
 13  max_date         dbdate 
 14  first_nps_date   dbdate 
 15  last_nps_date    dbdate 
 16  nb_fb            float64
 17  nb_fb_neg        float64
 18  nb_fb_pas        float64
 19  nb_fb_pos        float64
 20  fb_group         object 
 21  fb_nb_type       object 
dtypes: Int64(2), dbdate(8), float64(10), object(2)
memory usage: 331.5+ MB


In [75]:
print("Nb customers with purchase", df_s['user_id'].nunique())
print("Nb customers with purchase and nps", df_s_nps['user_id'].nunique())

Nb customers with purchase 8419907
Nb customers with purchase and nps 1097904


In [79]:
print('% of customers with purchases and NPS: ', (df_s_nps['user_id'].nunique()/df_s['user_id'].nunique())*100)

% of customers with purchases and NPS:  13.039383926687076


13 % of all customers that did purchases (web, store, both) gave NPS

In [81]:
def type_purchase(df):
    #customer did only web purchase
    if pd.isnull(df['min_date_store']) and df['min_date_web'] is not pd.NaT:
        return "only web"
    elif pd.isnull(df['min_date_web']) and df['min_date_store'] is not pd.NaT:
        return "only store"
    elif df['min_date_store'] is not pd.NaT and df['min_date_web'] is not pd.NaT:
        return "store, web"
    else:
        return "?"

In [82]:
df_s_6['purchase_type'] = df_s_6.apply(type_purchase, axis=1)

In [92]:
def last_purchase(df):
    if df['purchase_type'] == "store, web":
        if df['max_date_store'] >= df['max_date_web']:
            return "last store purchase"
        else:
            return "last web purchase" 
    else:
        return df['purchase_type']

In [93]:
df_s_6['last_purchase'] = df_s_6.apply(last_purchase, axis=1)

In [94]:
df_s_6.head(5)

,user_id,top_l_c,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,...,first_nps_date,last_nps_date,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,purchase_type,last_purchase
0,14,1,381.48,341.01,16.0,4.0,36.0,9.0,2022-02-12,2022-03-08,...,2022-05-04,2022-05-04,1.0,0.0,1.0,0.0,only passive,mono,"store, web",last store purchase
1,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2022-11-13,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last web purchase
2,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2022-11-13,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last web purchase
3,16,1,2394.90,NaN,51.0,NaN,162.0,NaN,2022-02-14,NaT,...,2023-02-07,2023-02-07,1.0,0.0,0.0,1.0,only positive,mono,only store,only store
4,19,1,4834.76,NaN,31.0,NaN,222.0,NaN,2022-05-05,NaT,...,2022-06-29,2022-11-15,2.0,0.0,1.0,1.0,passive and positive,mutli,only store,only store


In [95]:
def return_flag(df):
    # only web customer
    if df['purchase_type'] == 'only web':
        if df['last_nps_date'] >= df['max_date_web']:
            return "no return"
        else:
            return "return"
    # only store customer
    elif df['purchase_type'] == 'only store':
        if df['last_nps_date'] >= df['max_date_store']:
            return "no return"
        else:
            return "return"
    # store, web customer
    elif df['purchase_type'] == 'store, web':
        # last purchase web
        if df['last_purchase'] == 'last web purchase':
            if df['last_nps_date'] >= df['max_date_web']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_store']:
                    return "return/ store->web"
                else:
                    return "return/ ->store,web"
        #lasr purchase store
        if df['last_purchase'] == 'last store purchase':
            if df['last_nps_date'] >= df['max_date_store']:
                return "no return"
            else:
                if df['last_nps_date'] >= df['max_date_web']:
                    return "return/ web->store"
                else:
                    return "return/ ->web,store"

In [96]:
df_s_6['return_flag'] = df_s_6.apply(return_flag, axis=1)

/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/4113624140.py:27: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/4113624140.py:30: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_web']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/4113624140.py:10: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if df['last_nps_date'] >= df['max_date_store']:
/var/folders/t5/84slq0fn2gx51rghrcgxpjt80000gn/T/ipykernel_36486/4113624140

In [97]:
df_s_6.head()

,user_id,top_l_c,amount_to_store,amount_to_web,nb_carts_store,nb_carts_web,qty_store,qty_web,min_date_store,min_date_web,...,last_nps_date,nb_fb,nb_fb_neg,nb_fb_pas,nb_fb_pos,fb_group,fb_nb_type,purchase_type,last_purchase,return_flag
0,14,1,381.48,341.01,16.0,4.0,36.0,9.0,2022-02-12,2022-03-08,...,2022-05-04,1.0,0.0,1.0,0.0,only passive,mono,"store, web",last store purchase,"return/ ->web,store"
1,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last web purchase,"return/ ->store,web"
2,15,1,6672.94,940.28,59.0,10.0,340.0,110.0,2022-02-09,2022-01-03,...,2023-01-22,2.0,0.0,0.0,2.0,only positive,mutli,"store, web",last web purchase,"return/ ->store,web"
3,16,1,2394.90,NaN,51.0,NaN,162.0,NaN,2022-02-14,NaT,...,2023-02-07,1.0,0.0,0.0,1.0,only positive,mono,only store,only store,return
4,19,1,4834.76,NaN,31.0,NaN,222.0,NaN,2022-05-05,NaT,...,2022-11-15,2.0,0.0,1.0,1.0,passive and positive,mutli,only store,only store,return


In [100]:
df_s_6.to_parquet('./datasets/first_results.parquet')

In [101]:
df_s_nps_gr = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag'])['user_id'].nunique().reset_index()
df_s_nps_gr.columns = ['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag', 'nb_users']
df_s_nps_gr['share_%'] = (df_s_nps_gr['nb_users'] / df_s_nps_gr['nb_users'].sum()*100).round(2)

### Summary by NPS

In [ ]:
df_nps_summary_type = df_nps_4.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_nps_summary_type['share_%'] = (df_nps_summary_type['user_id'] / df_nps_summary_type['user_id'].sum())*100
df_nps_summary_type

,fb_nb_type,user_id,share_%
0,mono,889117,72.614886
1,mutli,335311,27.385114


In [ ]:
df_nps_summary_group = df_nps_4.groupby('fb_group')['user_id'].nunique().reset_index()
df_nps_summary_group['share_%'] = (df_nps_summary_group['user_id'] / df_nps_summary_group['user_id'].sum())*100
df_nps_summary_group = df_nps_summary_group.sort_values(by='share_%', ascending=False)
df_nps_summary_group

,fb_group,user_id,share_%
5,only positive,680303,55.560882
4,only passive,277548,22.667564
3,only negative,129318,10.561503
6,passive and positive,77828,6.356274
1,negative and positive,26855,2.193269
0,negative and passive,21942,1.792020
2,"negative, passive, positive",10634,0.868487


In [ ]:
df_nps_summary = df_nps_4.groupby(['fb_nb_type', 'fb_group'])['user_id'].nunique().reset_index()
df_nps_summary['share_%'] = (df_nps_summary['user_id'] / df_nps_summary['user_id'].sum())*100
df_nps_summary = df_nps_summary.sort_values(by=['fb_nb_type','share_%'], ascending=[True, False])
df_nps_summary

,fb_nb_type,fb_group,user_id,share_%
2,mono,only positive,524428,42.830448
1,mono,only passive,246769,20.153819
0,mono,only negative,117920,9.630619
8,mutli,only positive,155875,12.730434
9,mutli,passive and positive,77828,6.356274
7,mutli,only passive,30779,2.513745
4,mutli,negative and positive,26855,2.193269
3,mutli,negative and passive,21942,1.792020
6,mutli,only negative,11398,0.930884
5,mutli,"negative, passive, positive",10634,0.868487


### Summary by Sales

In [103]:
df_s_nps_mm = df_s_6.groupby('fb_nb_type')['user_id'].nunique().reset_index()
df_s_nps_mm.columns = ['fb_nb_type', 'nb_users']
df_s_nps_mm['share_%'] = (df_s_nps_mm['nb_users'] / df_s_nps_mm['nb_users'].sum()*100).round(2)
df_s_nps_mm

,fb_nb_type,nb_users,share_%
0,mono,768167,69.97
1,mutli,329718,30.03


In [104]:
df_s_nps_mm_t = df_s_6.groupby(['fb_nb_type', 'purchase_type'])['user_id'].nunique().reset_index()
df_s_nps_mm_t.columns = ['fb_nb_type', 'purchase_type', 'nb_users']
df_s_nps_mm_t['share_%'] = (df_s_nps_mm_t['nb_users'] / df_s_nps_mm_t['nb_users'].sum()*100).round(2)
df_s_nps_mm_t

,fb_nb_type,purchase_type,nb_users,share_%
0,mono,only store,298649,27.20
1,mono,only web,204069,18.59
2,mono,"store, web",265449,24.18
3,mutli,only store,140917,12.84
4,mutli,only web,37058,3.38
5,mutli,"store, web",151743,13.82


In [105]:
df_s_nps_lp = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase'])['user_id'].nunique().reset_index()
df_s_nps_lp.columns = ['fb_nb_type', 'purchase_type', 'last_purchase', 'nb_users']
df_s_nps_lp['share_%'] = (df_s_nps_lp['nb_users'] / df_s_nps_lp['nb_users'].sum()*100).round(2)
df_s_nps_lp

,fb_nb_type,purchase_type,last_purchase,nb_users,share_%
0,mono,only store,only store,298649,27.20
1,mono,only web,only web,204069,18.59
2,mono,"store, web",last store purchase,209022,19.04
3,mono,"store, web",last web purchase,56427,5.14
4,mutli,only store,only store,140917,12.84
5,mutli,only web,only web,37058,3.38
6,mutli,"store, web",last store purchase,121871,11.10
7,mutli,"store, web",last web purchase,29872,2.72


In [106]:
df_s_nps_gr = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag'])['user_id'].nunique().reset_index()
df_s_nps_gr.columns = ['fb_nb_type', 'purchase_type', 'last_purchase', 'return_flag', 'nb_users']
df_s_nps_gr['share_%'] = (df_s_nps_gr['nb_users'] / df_s_nps_gr['nb_users'].sum()*100).round(2)
df_s_nps_gr

,fb_nb_type,purchase_type,last_purchase,return_flag,nb_users,share_%
0,mono,only store,only store,no return,66776,6.08
1,mono,only store,only store,return,231873,21.12
2,mono,only web,only web,no return,56512,5.15
3,mono,only web,only web,return,147557,13.44
4,mono,"store, web",last store purchase,no return,12076,1.10
5,mono,"store, web",last store purchase,"return/ ->web,store",138969,12.66
6,mono,"store, web",last store purchase,return/ web->store,57977,5.28
7,mono,"store, web",last web purchase,no return,8065,0.73
8,mono,"store, web",last web purchase,"return/ ->store,web",25360,2.31
9,mono,"store, web",last web purchase,return/ store->web,23002,2.10


### Summary Total

In [107]:
df_s_nps_res = df_s_6.groupby(['fb_nb_type', 'purchase_type', 'last_purchase', 'fb_group',\
                                'return_flag'])['user_id'].nunique().reset_index()
df_s_nps_res.columns = ['fb_nb_type', 'purchase_type', 'last_purchase','fb_group', 'return_flag', 'nb_users']
df_s_nps_res['share_%'] = (df_s_nps_res['nb_users'] / df_s_nps_res['nb_users'].sum()*100).round(2)
df_s_nps_res

,fb_nb_type,purchase_type,last_purchase,fb_group,return_flag,nb_users,share_%
0,mono,only store,only store,only negative,no return,10192,0.93
1,mono,only store,only store,only negative,return,35337,3.22
2,mono,only store,only store,only passive,no return,16965,1.55
3,mono,only store,only store,only passive,return,60926,5.55
4,mono,only store,only store,only positive,no return,39619,3.61
...,...,...,...,...,...,...,...
95,mutli,"store, web",last web purchase,only positive,"return/ ->store,web",5763,0.52
96,mutli,"store, web",last web purchase,only positive,return/ store->web,5470,0.50
97,mutli,"store, web",last web purchase,passive and positive,no return,1357,0.12
98,mutli,"store, web",last web purchase,passive and positive,"return/ ->store,web",2781,0.25


In [109]:
df_s_nps_res[df_s_nps_res['return_flag']=='no return'].sort_values(by='share_%', ascending=False)

,fb_nb_type,purchase_type,last_purchase,fb_group,return_flag,nb_users,share_%
4,mono,only store,only store,only positive,no return,39619,3.61
10,mono,only web,only web,only positive,no return,35221,3.21
2,mono,only store,only store,only passive,no return,16965,1.55
8,mono,only web,only web,only passive,no return,14861,1.35
40,mutli,only store,only store,only positive,no return,11824,1.08
0,mono,only store,only store,only negative,no return,10192,0.93
54,mutli,only web,only web,only positive,no return,8309,0.76
18,mono,"store, web",last store purchase,only positive,no return,7220,0.66
6,mono,only web,only web,only negative,no return,6430,0.59
42,mutli,only store,only store,passive and positive,no return,6033,0.55
